In [ ]:
!pip install torch==2.4
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cpu.html

In [ ]:
import torch
import torch_geometric
from torch_geometric.datasets import Planetoid

use_cuda_if_available = False

print(torch.__version__)

2.4.0


# Load the dataset

In [4]:
dataset = Planetoid(root="tutorial1",name= "Cora")

Processing...
Done!


##### Dataset properties

In [5]:
print(dataset)
print("number of graphs:\t\t",len(dataset))
print("number of classes:\t\t",dataset.num_classes)  # this is a classification task (7 classes)
print("number of node features:\t",dataset.num_node_features)  # EACH node has 1433 features
print("number of edge features:\t",dataset.num_edge_features)

Cora()
number of graphs:		 1
number of classes:		 7
number of node features:	 1433
number of edge features:	 0


##### Dataset shapes

In [6]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


/opt/homebrew/Caskroom/miniforge/base/envs/pyg/lib/python3.12/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [ ]:
print("edge_index:\t\t",dataset.data.edge_index.shape)
print(dataset.data.edge_index)  # edge_index shape = (2, num_edges), the two rows represent the source and target nodes of each edge
print("\n")
print("train_mask:\t\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask) # also test_mask and val_mask
print("\n")
print("x:\t\t",dataset.data.x.shape)  
print(dataset.data.x)  # node features shape = (num_nodes, num_node_features)
print("\n")
print("y:\t\t",dataset.data.y.shape)
print(dataset.data.y)  # "true" labels for each node, shape = (num_nodes,)

edge_index:		 torch.Size([2, 10556])
tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


In [9]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv  # covered in the lecture slides

In [10]:
data = dataset[0]  # because we only have one graph in the dataset
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [11]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = SAGEConv(
            dataset.num_features,
            dataset.num_classes,
            aggr="max"
        ) # max, mean, add, ... (any order invariant aggregation function)

    def forward(self):
        x = self.conv(data.x, data.edge_index)  # we update the node features using the SAGEConv layer
        return F.log_softmax(x, dim=1)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [13]:
device

device(type='cpu')

In [14]:
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()

def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [15]:
best_val_acc = test_acc = 0
for epoch in range(1,100):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    if epoch % 10 == 0:
        print(log.format(epoch, best_val_acc, test_acc))

Epoch: 010, Val: 0.7300, Test: 0.7240
Epoch: 020, Val: 0.7320, Test: 0.7060
Epoch: 030, Val: 0.7320, Test: 0.7060
Epoch: 040, Val: 0.7320, Test: 0.7060
Epoch: 050, Val: 0.7320, Test: 0.7060
Epoch: 060, Val: 0.7360, Test: 0.7140
Epoch: 070, Val: 0.7360, Test: 0.7140
Epoch: 080, Val: 0.7360, Test: 0.7140
Epoch: 090, Val: 0.7360, Test: 0.7140
